In [4]:
import sys
sys.path.append('../../')
sys.path.append('../../../flaming-choripan')

In [90]:
%load_ext autoreload
%autoreload 2
from flamingchoripan.files import load_pickle, get_filedirs
from flamingchoripan.datascience.statistics import XError
import numpy as np

class Rank():
    def __init__(self, name, print_n):
        self.name = name
        self.print_n = print_n
        self.names = []
        self.values = []
        
    def add(self, name, value):
        self.names.append(name)
        self.values.append(value)
        
    def calcule_rank(self):
        self.idxs = np.argsort(self.values)[::-1].tolist()
        
    def __len__(self):
        return len(self.names)

    def __repr__(self):
        txt = f'{self.name}:\n'
        idxs = self.idxs
        for k,idx in enumerate(idxs):
            txt += f'({k+1}) - {self.names[idx]}: {self.values[idx]}\n'
            if k+1>=self.print_n:
                break
        return txt[:-1]
    
rootdir = '/home/opimentel/tesis/astro-lightcurves-handler/save/alerceZTFv5.1'
load_rootdir = f'{rootdir}/{method}'
filedirs = get_filedirs(load_rootdir, fext='ferror')
band_names = ['g', 'r']

results_dict = {}
for method in ['curve_fit']:
    results_dict[method] = {
        'mb-fit-error':[],
        'mb-null-fits':[],
    }
    for b in band_names:
        results_dict[method].update({
            f'{b}-fit-error':[],
            f'{b}-null-fits':[],
        })

method = 'curve_fit'
rank = Rank('mb-rank', 10)
rank_bdict = {b:Rank(f'{b}-rank', 10) for b in band_names}

for filedir in filedirs:
    fdict = load_pickle(filedir, verbose=0)
    lcobj_name = fdict['lcobj_name']
    xes = []
    for b in band_names:
        ov = fdict['error_bdict'][b]
        v = [x for x in ov if x>0]
        if len(v)>0:
            xe = XError(v, 0)
            rank_bdict[b].add(lcobj_name, xe.mean)
            xes.append(xe)
            results_dict[method][f'{b}-fit-error'] += v
            results_dict[method][f'{b}-null-fits'].append(1-len(v)/len(ov))
            results_dict[method][f'mb-fit-error'] += v
            results_dict[method][f'mb-null-fits'].append(1-len(v)/len(ov))
    if len(xes)>0:
        rank.add(lcobj_name, np.mean([xe.mean for xe in xes]))

for method in ['curve_fit']:
    results_dict[method]['mb-fit-error'] = XError(results_dict[method]['mb-fit-error'], 0)
    results_dict[method]['mb-null-fits'] = np.mean(results_dict[method]['mb-null-fits'])*100
    for b in band_names:
        results_dict[method][f'{b}-fit-error'] = XError(results_dict[method][f'{b}-fit-error'], 0)
        results_dict[method][f'{b}-null-fits'] = np.mean(results_dict[method][f'{b}-null-fits'])*100

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
for method in ['curve_fit']:
    print(results_dict[method])

{'mb-fit-error': XError(2.53+-15.66, dim=0), 'mb-null-fits': 0.9852372139973081, 'g-fit-error': XError(2.27+-12.09, dim=0), 'g-null-fits': 1.7850719424460433, 'r-fit-error': XError(2.75+-18.18, dim=0), 'r-null-fits': 0.2824747155499368}


In [72]:
rank.calcule_rank()
print(rank)
for b in band_names:
    rank_bdict[b].calcule_rank()
    print(rank_bdict[b])

mb-rank:
(1) - ZTF19aajxwnz: 71.73245239257812
(2) - ZTF19aacgslb: 52.663902282714844
(3) - ZTF18achaqmd: 43.40021514892578
(4) - ZTF18abltdkd: 20.69577980041504
(5) - ZTF18acrnfww: 19.955039978027344
(6) - ZTF19aatlmbo: 16.186267852783203
(7) - ZTF18acslpba: 15.137617111206055
(8) - ZTF19aawqckx: 13.417102813720703
(9) - ZTF18acvhggp: 12.587767601013184
(10) - ZTF19abdzehg: 11.635040283203125
g-rank:
(1) - ZTF19aajxwnz: 71.73245239257812
(2) - ZTF18achaqmd: 43.40021514892578
(3) - ZTF18acrnfww: 38.3190803527832
(4) - ZTF18acslpba: 28.983369827270508
(5) - ZTF19aatlmbo: 28.553916931152344
(6) - ZTF19aacgslb: 22.98574447631836
(7) - ZTF19abdzehg: 22.46877670288086
(8) - ZTF18abltdkd: 20.69577980041504
(9) - ZTF18acemhyb: 18.534881591796875
(10) - ZTF18accngee: 16.9926700592041
r-rank:
(1) - ZTF19aacgslb: 82.34205627441406
(2) - ZTF19acbunmk: 20.378925323486328
(3) - ZTF18acvhggp: 12.587767601013184
(4) - ZTF18abcflnz: 11.076953887939453
(5) - ZTF19aafncsv: 10.000890731811523
(6) - ZTF19